
<pre>
  ____                      _                              _     _               
 |  _ \                    | |                            | |   (_)              
 | |_) |  ___  _ __    ___ | |__   _ __ ___    __ _  _ __ | | __ _  _ __    __ _ 
 |  _ <  / _ \| '_ \  / __|| '_ \ | '_ ` _ \  / _` || '__|| |/ /| || '_ \  / _` |
 | |_) ||  __/| | | || (__ | | | || | | | | || (_| || |   |   < | || | | || (_| |
 |____/  \___||_| |_| \___||_| |_||_| |_| |_| \__,_||_|   |_|\_\|_||_| |_| \__, |
                                                                            __/ |
                                                                           |___/ 
</pre>

# Imports

In [1]:
import torch
import mmcv
# import mmseg
from mmseg.apis import inference_segmentor
from mmseg.apis import init_segmentor
from mmseg.ops import resize

import mmcv
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import glob

from tqdm import tqdm

from glob import glob

import matplotlib.image as mpimg

import numpy as np

sys.path.append("/project_ghent/woverbie/code/SegVit/decode_heads")
sys.path.append("/project_ghent/woverbie/code/SegVit/losses")

import atm_head, tpn_atm_head
import atm_loss

import torch.nn as nn



import cv2 as cv

import math
from typing import Callable, Tuple

import torch

from scipy.ndimage import zoom

/opt/conda/lib/python3.11/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


# Constants

In [2]:
# VIDEO_MUNICH
# VIDEO_INGOLSTADT
VIDEO_GAIMERSHEIM = sorted(glob('../dataset/camera_lidar/20180810_150607/camera/cam_front_center/*.png'))


In [3]:
config_file = '../configs/token_reducing_config_gpu.py'
checkpoint_file = '../model/checkpoint/COCOstuff_49.9.pth'
model_og = init_segmentor(config_file, checkpoint_file, device='cuda:0') # can not be used for time measures!

for i, block in enumerate(model_og.backbone.layers):
    model_og.backbone.layers[i].r = 1

Initializing ATM head
load checkpoint from local path: ../model/checkpoint/COCOstuff_49.9.pth


## Fixed r

In [4]:
config_file = '../configs/token_reducing_config_gpu.py'
checkpoint_file = '../model/checkpoint/COCOstuff_49.9.pth'

r_values = np.arange(0.98, 0.82, -0.02).round(2).tolist()

print(r_values)

for r_value in r_values:
    print(r_value)

    # model
    model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

    # Overwriting the r values
    for i, block in enumerate(model.backbone.layers):
        model.backbone.layers[i].r = r_value
        model.backbone.layers[i].reduction_interval = 1

    pixel_wise_acc_list = []
    
    prunes = []

    reduced_tokens_heatmap = None
    
    # Display the image
    for i, img_path in enumerate(VIDEO_GAIMERSHEIM):
    
        print('image', i)
        
        result = inference_segmentor(model, img_path)
        result_og = inference_segmentor(model_og, img_path)
    
        result1 = result[0] - result_og[0]
        binary_result = (result1 != 0).astype(int)
        
        percentage_ones = (binary_result.sum() / binary_result.size) * 100
        pixel_wise_acc_list.append(percentage_ones)
    
        reduction = []
    
        for i, layer in enumerate(model.backbone.layers):
    
            if layer.unm_idx is not None:
                
                if layer.src_idx is not None:
                    reduction.append(layer.src_idx.shape[1])
                else:
                    reduction.append(0)
    
        prunes.append(reduction)

        pruned_tokens = []
        for i, layer in reversed(list(enumerate(model.backbone.layers))):
            if layer.unm_idx is not None:
                if len(pruned_tokens) == 0:
                    pruned_tokens = [0] * len(layer.unm_idx[0])
    
                if layer.src_idx is not None:
                    temp = pruned_tokens
    
                    pruned_tokens = [0] * ( len(layer.unm_idx[0]) + len(layer.src_idx[0]))
    
                    for i, unm_idx in enumerate(layer.unm_idx[0]):
                        pruned_tokens[unm_idx] = temp[i]
    
                    for i, src_idx in enumerate(layer.src_idx[0]):
                        pruned_tokens[src_idx] = 1
    
        if len(pruned_tokens) == 0:
            pruned_tokens = [0]*1024
            
        tokens_reduced = np.array(pruned_tokens).reshape((32, 32))
        
        # scaled_tokens_reduced = np.repeat(np.repeat(tokens_reduced, 16, axis=0), 16, axis=1)
        zoom_factor = 1208 / 32
        scaled_tokens_reduced = zoom(tokens_reduced, zoom=(zoom_factor, zoom_factor), order=1)

        if reduced_tokens_heatmap is not None:
            reduced_tokens_heatmap += scaled_tokens_reduced
        else:
            reduced_tokens_heatmap = scaled_tokens_reduced

    # prunes
    np.save(f"benchmarking/tokens_pruned_all_layers_{r_value}.npy", prunes)
    
    # pixel_wise_acc_list
    np.save(f"benchmarking/pixel_wise_acc_list_all_layers_{r_value}.npy", pixel_wise_acc_list)

    # encode times
    encode_times_np = np.array(model.encode_times)
    np.save(f"benchmarking/encode_times_all_layers_{r_value}.npy", encode_times_np)

    # heatmap
    np.save(f"benchmarking/reduced_tokens_heatmap_all_layers_{r_value}.npy", reduced_tokens_heatmap)
    

[0.98, 0.96, 0.94, 0.92, 0.9, 0.88, 0.86, 0.84, 0.82]
0.98
Initializing ATM head
load checkpoint from local path: ../model/checkpoint/COCOstuff_49.9.pth
image 0


FileNotFoundError: [Errno 2] No such file or directory: '/project_ghent/woverbie/benchmarking/SegVit_test_2/test.txt'

## Linear r

In [ ]:
# model
model = init_segmentor(config_file, checkpoint_file, device='cuda:0')



r_values = np.linspace(0.995, 0.93, len(model.backbone.layers)).tolist()
for i, block in enumerate(model.backbone.layers):
    # Assuming the transformer blocks are in backbone.layers
    model.backbone.layers[i].r = r_values[i]


pixel_wise_acc_list = []

prunes = []

reduced_tokens_heatmap = None

# Display the image
for i, img_path in enumerate(VIDEO_GAIMERSHEIM):

    print('image', i)
    
    result = inference_segmentor(model, img_path)
    result_og = inference_segmentor(model_og, img_path)

    result1 = result[0] - result_og[0]
    binary_result = (result1 != 0).astype(int)
    
    percentage_ones = (binary_result.sum() / binary_result.size) * 100
    pixel_wise_acc_list.append(percentage_ones)

    reduction = []

    for i, layer in enumerate(model.backbone.layers):

        if layer.unm_idx is not None:
            
            if layer.src_idx is not None:
                reduction.append(layer.src_idx.shape[1])
            else:
                reduction.append(0)

    prunes.append(reduction)

    pruned_tokens = []
    for i, layer in reversed(list(enumerate(model.backbone.layers))):
        if layer.unm_idx is not None:
            if len(pruned_tokens) == 0:
                pruned_tokens = [0] * len(layer.unm_idx[0])

            if layer.src_idx is not None:
                temp = pruned_tokens

                pruned_tokens = [0] * ( len(layer.unm_idx[0]) + len(layer.src_idx[0]))

                for i, unm_idx in enumerate(layer.unm_idx[0]):
                    pruned_tokens[unm_idx] = temp[i]

                for i, src_idx in enumerate(layer.src_idx[0]):
                    pruned_tokens[src_idx] = 1

    if len(pruned_tokens) == 0:
        pruned_tokens = [0]*1024
        
    tokens_reduced = np.array(pruned_tokens).reshape((32, 32))
    
    # scaled_tokens_reduced = np.repeat(np.repeat(tokens_reduced, 16, axis=0), 16, axis=1)
    zoom_factor = 1208 / 32
    scaled_tokens_reduced = zoom(tokens_reduced, zoom=(zoom_factor, zoom_factor), order=1)

    if reduced_tokens_heatmap is not None:
        reduced_tokens_heatmap += scaled_tokens_reduced
    else:
        reduced_tokens_heatmap = scaled_tokens_reduced

# prunes
np.save(f"benchmarking/tokens_pruned_lin_0.995_0.93.npy", prunes)

# pixel_wise_acc_list
np.save(f"benchmarking/pixel_wise_acc_list_lin_0.995_0.93.npy", pixel_wise_acc_list)

# encode times
encode_times_np = np.array(model.encode_times)
np.save(f"benchmarking/encode_times_lin_0.995_0.93.npy", encode_times_np)

# heatmap
np.save(f"benchmarking/reduced_tokens_heatmap_lin_0.995_0.93.npy", reduced_tokens_heatmap)

## amount of reduction layers

In [ ]:
config_file = '../configs/token_reducing_config_gpu.py'
checkpoint_file = '../model/checkpoint/COCOstuff_49.9.pth'

# r_values = np.arange(1, 0.60, -0.02).round(2).tolist()

# print(r_values)

deviders_of_24 = [24, 12, 8, 6, 4, 3, 2, 1]

for reduction_interval in deviders_of_24:
    # print(r_value)

    # model
    model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

    # Overwriting the r values
    r_values = np.linspace(0.995, 0.93, len(model.backbone.layers)).tolist()
    for i, block in enumerate(model.backbone.layers):
        model.backbone.layers[i].r = r_values[i]
        model.backbone.layers[i].reduction_interval = reduction_interval

        # print(model.backbone.layers[i].reduction_interval)

    pixel_wise_acc_list = []
    
    prunes = []

    reduced_tokens_heatmap = None
    
    # Display the image
    for i, img_path in enumerate(VIDEO_GAIMERSHEIM):
    
        print('image', i)
        
        result = inference_segmentor(model, img_path)
        result_og = inference_segmentor(model_og, img_path)
    
        result1 = result[0] - result_og[0]
        binary_result = (result1 != 0).astype(int)
        
        percentage_ones = (binary_result.sum() / binary_result.size) * 100
        pixel_wise_acc_list.append(percentage_ones)
    
        reduction = []
    
        for i, layer in enumerate(model.backbone.layers):
    
            if layer.unm_idx is not None:
                
                if layer.src_idx is not None:
                    reduction.append(layer.src_idx.shape[1])
                else:
                    reduction.append(0)
    
        prunes.append(reduction)

        pruned_tokens = []
        for i, layer in reversed(list(enumerate(model.backbone.layers))):
            if layer.unm_idx is not None:
                if len(pruned_tokens) == 0:
                    pruned_tokens = [0] * len(layer.unm_idx[0])
    
                if layer.src_idx is not None:
                    temp = pruned_tokens
    
                    pruned_tokens = [0] * ( len(layer.unm_idx[0]) + len(layer.src_idx[0]))
    
                    for i, unm_idx in enumerate(layer.unm_idx[0]):
                        pruned_tokens[unm_idx] = temp[i]
    
                    for i, src_idx in enumerate(layer.src_idx[0]):
                        pruned_tokens[src_idx] = 1
    
        if len(pruned_tokens) == 0:
            pruned_tokens = [0]*1024
            
        tokens_reduced = np.array(pruned_tokens).reshape((32, 32))
        
        # scaled_tokens_reduced = np.repeat(np.repeat(tokens_reduced, 16, axis=0), 16, axis=1)
        zoom_factor = 1208 / 32
        scaled_tokens_reduced = zoom(tokens_reduced, zoom=(zoom_factor, zoom_factor), order=1)

        if reduced_tokens_heatmap is not None:
            reduced_tokens_heatmap += scaled_tokens_reduced
        else:
            reduced_tokens_heatmap = scaled_tokens_reduced

    # prunes
    np.save(f"benchmarking/tokens_pruned_lin_int_{reduction_interval}.npy", prunes)
    
    # pixel_wise_acc_list
    np.save(f"benchmarking/pixel_wise_acc_list_lin_int_{reduction_interval}.npy", pixel_wise_acc_list)

    # encode times
    encode_times_np = np.array(model.encode_times)
    np.save(f"benchmarking/encode_times_lin_int_{reduction_interval}.npy", encode_times_np)

    # heatmap
    np.save(f"benchmarking/reduced_tokens_heatmap_lin_int_{reduction_interval}.npy", reduced_tokens_heatmap)
    

## OG model

In [ ]:
from tqdm import tqdm
config_file = '/project_ghent/woverbie/code/SegVit/configs/segvit/segvit_vit-l_jax_512x512_80k_cocostuff10k.py'
checkpoint_file = '../../benchmarking/SegVit_test_2/models/COCOstuff_49.9.pth'
model_og = init_segmentor(config_file, checkpoint_file, device='cuda:0')

In [ ]:

square_imgs = sorted(glob('../dataset/squares/*.png'))

for img in tqdm(square_imgs):
    result = inference_segmentor(model_og, img)


np.save(f"benchmarking/model_og_encode_times_gpu.npy", model_og.encode_times)
np.save(f"benchmarking/model_og_decode_times_gpu.npy", model_og.decode_times)


In [ ]:
# flops
import cv2
import torch
from torch.profiler import profile, ProfilerActivity
from mmseg.datasets.pipelines import Compose

cfg = model_og.cfg  # The config file used to initialize the model
test_pipeline = Compose(cfg.data.test.pipeline)

def profile_inference(model, img):
    with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], 
                 with_flops=True,  # Enable FLOP counting
                 record_shapes=True) as prof:
        # Perform inference
        _ = inference_segmentor(model_og, img)

    return prof

flop_list = []

# Display the image

square_imgs = sorted(glob('../dataset/squares/*.png'))

for i, img_path in tqdm(enumerate(square_imgs)):

    prof = profile_inference(model_og, img_path)
    
    avg = sum(op.flops for op in prof.key_averages() if hasattr(op, 'flops'))

    flop_list.append(avg)


print(np.median(flop_list))
print(np.mean(flop_list))

In [ ]:
print(np.median(flop_list))
print(np.mean(flop_list))

# CPU TIMES

In [ ]:
from tqdm import tqdm

config_file_cpu = '/project_ghent/woverbie/code/configs/config.py'
checkpoint_file = '../../benchmarking/SegVit_test_2/models/COCOstuff_49.9.pth'

model_cpu = init_segmentor(config_file_cpu, checkpoint_file, device='cpu')

# Overwriting the r values
r_values = np.linspace(0.995, 0.93, len(model_cpu.backbone.layers)).tolist()
for i, block in enumerate(model_cpu.backbone.layers):
    model_cpu.backbone.layers[i].r = r_values[i]
    model_cpu.backbone.layers[i].reduction_interval = 8



test_set = VIDEO_GAIMERSHEIM[1000:2000]

# Display the image
for i, img_path in tqdm(enumerate(test_set)):
    result = inference_segmentor(model_cpu, img_path)


encode_times_cpu = np.array(model_cpu.encode_times)
np.save(f"benchmarking/final_token_reducing_cpu_encode_times.npy", encode_times_cpu)

decode_times_cpu = np.array(model_cpu.decode_times)
np.save(f"benchmarking/final_token_reducing_cpu_decode_times.npy", decode_times_cpu)

In [ ]:
config_file_og = '/project_ghent/woverbie/code/SegVit/configs/segvit/segvit_vit-l_jax_512x512_80k_cocostuff10k.py'
checkpoint_file = '../../benchmarking/SegVit_test_2/models/COCOstuff_49.9.pth'
model_og_cpu = init_segmentor(config_file_og, checkpoint_file, device='cpu')

test_set = VIDEO_GAIMERSHEIM[1000:2000]

# Display the image
for i, img_path in tqdm(enumerate(test_set)):
    result = inference_segmentor(model_og_cpu, img_path)

encode_times_og_cpu = np.array(model_og_cpu.encode_times)
np.save(f"benchmarking/og_cpu_encode_times.npy", encode_times_og_cpu)

decode_times_og_cpu = np.array(model_og_cpu.decode_times)
np.save(f"benchmarking/og_cpu_decode_times.npy", decode_times_og_cpu)

In [ ]:
np.load('benchmarking/og_cpu_encode_times.npy')

In [ ]:
# Final Model

# FLOPS FINAL MODEL

In [ ]:
config_file = '/project_ghent/woverbie/code/configs/token_reducing_config_gpu.py'
checkpoint_file = '../../benchmarking/SegVit_test_2/models/COCOstuff_49.9.pth'
model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

# Setting the r values

r_values = np.linspace(0.995, 0.93, len(model.backbone.layers)).tolist()
for i, block in enumerate(model.backbone.layers):
    # Assuming the transformer blocks are in backbone.layers
    model.backbone.layers[i].r = r_values[i]
    model.backbone.layers[i].reduction_interval = 8

In [ ]:

## Test with an inference function
import cv2
import torch
from torch.profiler import profile, ProfilerActivity
from mmseg.datasets.pipelines import Compose
from tqdm import tqdm

cfg = model.cfg  # The config file used to initialize the model
test_pipeline = Compose(cfg.data.test.pipeline)

prof_list = []

def profile_inference(model, img):
    with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], 
                 with_flops=True,  # Enable FLOP counting
                 record_shapes=True) as prof:
        # Perform inference
        _ = inference_segmentor(model, img)
        
    return prof



# Display the image
for i, img_path in tqdm(enumerate(VIDEO_GAIMERSHEIM)):

    prof = profile_inference(model, img_path)
    
    avg = sum(op.flops for op in prof.key_averages() if hasattr(op, 'flops'))

    print(avg)

    prof_list.append(avg)



np.save(f"benchmarking/final_model_flops.npy", prof_list)

# VIDEO

In [ ]:
def center_crop(img_array, crop_size):
    """
    Crop the center of the image.
    
    Parameters:
        img_array (numpy.ndarray): Input image as a NumPy array.
        crop_size (tuple): Desired crop size (height, width).
    
    Returns:
        numpy.ndarray: Center-cropped image.
    """
    h, w, *channels = img_array.shape
    crop_height, crop_width = crop_size

    # Calculate the coordinates for the crop
    start_y = (h - crop_height) // 2
    start_x = (w - crop_width) // 2

    # Perform the crop
    cropped_img = img_array[start_y:start_y + crop_height, start_x:start_x + crop_width]

    return cropped_img

In [ ]:
# model
from PIL import Image
from tqdm import tqdm

config_file = '../configs/token_reducing_config_gpu.py'
checkpoint_file = '../model/checkpoint/COCOstuff_49.9.pth'

model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

r_values = np.linspace(0.995, 0.93, len(model.backbone.layers)).tolist()
for i, block in enumerate(model.backbone.layers):
    # Assuming the transformer blocks are in backbone.layers
    model.backbone.layers[i].r = r_values[i]


pixel_wise_acc_list = [0] * 150

prunes = [[0,0,0]] * 150

flop_list = [0] * 150

# Load the NumPy array
flop_array = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/final_model_flops.npy')

# Concatenate the list and NumPy array
combined = np.concatenate((np.array(flop_list), flop_array))

reduced_tokens_heatmap = None

# Display the image
for image_id_, img_path in tqdm(enumerate(VIDEO_GAIMERSHEIM)):

    # print('image', image_id_)
    
    result = inference_segmentor(model, img_path)
    result_og = inference_segmentor(model_og, img_path)

    result1 = result[0] - result_og[0]
    binary_result = (result1 != 0).astype(int)
    
    percentage_ones = (binary_result.sum() / binary_result.size) * 100
    pixel_wise_acc_list.append(percentage_ones)

    reduction = []

    for i, layer in enumerate(model.backbone.layers):

        if layer.unm_idx is not None:
            
            if layer.src_idx is not None:
                reduction.append(layer.src_idx.shape[1])
            else:
                reduction.append(0)

    prunes.append(reduction)

    pruned_tokens = []
    for i, layer in reversed(list(enumerate(model.backbone.layers))):
        if layer.unm_idx is not None:
            if len(pruned_tokens) == 0:
                pruned_tokens = [0] * len(layer.unm_idx[0])

            if layer.src_idx is not None:
                temp = pruned_tokens

                pruned_tokens = [0] * ( len(layer.unm_idx[0]) + len(layer.src_idx[0]))

                for i, unm_idx in enumerate(layer.unm_idx[0]):
                    pruned_tokens[unm_idx] = temp[i]

                for i, src_idx in enumerate(layer.src_idx[0]):
                    pruned_tokens[src_idx] = 1

    if len(pruned_tokens) == 0:
        pruned_tokens = [0]*1024
        
    tokens_reduced = np.array(pruned_tokens).reshape((32, 32))
    
    # scaled_tokens_reduced = np.repeat(np.repeat(tokens_reduced, 16, axis=0), 16, axis=1)
    zoom_factor = 1208 / 32
    scaled_tokens_reduced = zoom(tokens_reduced, zoom=(zoom_factor, zoom_factor), order=1)

    if reduced_tokens_heatmap is not None:
        reduced_tokens_heatmap += scaled_tokens_reduced
    else:
        reduced_tokens_heatmap = scaled_tokens_reduced


    fig, axs = plt.subplots(2, 4, figsize=(16, 8))

    image = Image.open(img_path)
    image_array = np.array(image)
    cropped_image = center_crop(image_array, (1208, 1208))
    axs[0, 0].imshow(cropped_image)
    axs[0, 0].axis('off')
    axs[0, 0].set_title('Input Image')
    
    axs[0, 1].imshow(result[0])
    axs[0, 1].axis('off')
    axs[0, 1].set_title('Result ToRe')

    axs[0, 2].imshow(result_og[0])
    axs[0, 2].axis('off')
    axs[0, 2].set_title('Result Original')
    
    tokens_pruned_sum = np.sum(prunes, axis=1)
    axs[1, 0].plot(tokens_pruned_sum[-150:], label='')
    axs[1, 0].set_title('Tokens Pruned')
    axs[1, 0].set_ylim(0, 1024)
    axs[1, 0].legend()
    axs[1, 0].grid()

    acc = [100 - item for item in pixel_wise_acc_list]
    axs[1, 1].plot(acc[-150:], label='')
    axs[1, 1].set_title('Pixel-Wise Accuracy')
    axs[1, 1].set_ylim(0, 100)
    axs[1, 1].legend()
    axs[1, 1].grid()
    
    axs[1, 3].imshow(scaled_tokens_reduced)
    axs[1, 3].axis('off')
    axs[1, 3].set_title('Tokens Reduced')
    
    axs[0, 3].imshow(binary_result)
    axs[0, 3].axis('off')
    axs[0, 3].set_title('Diff ToRe and Original')


    axs[1, 2].plot(combined[image_id_:image_id_+150], label='')
    axs[1, 2].set_title('FLOPs')
    axs[1, 2].set_ylim(0, 800000000000)
    axs[1, 2].grid()
    
    # Adjust layout for better spacing
    plt.tight_layout()

    plt.savefig(f"video/visualisation_{image_id_}.png")
    plt.show()
    

In [ ]:
import cv2

output_file = "output_video.mp4"
frame_rate = 30  # Frames per second

# Use glob to get all PNG files in the folder
image_files = glob("video/*.png")
image_files.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))  # Sort by numeric order in filename


# Read the first image to get the dimensions
first_image = cv2.imread(image_files[0])
height, width, _ = first_image.shape

# Define the video codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
video_writer = cv2.VideoWriter(output_file, fourcc, frame_rate, (width, height))

# Loop through the sorted images and write them to the video
for image_path in image_files:
    frame = cv2.imread(image_path)
    video_writer.write(frame)

# Release the VideoWriter object
video_writer.release()
print(f"Video saved as {output_file}")